In [1]:
# installation
!pip install transformers accelerate datasets outlines

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# libraries
import os
import re
from google.colab import files
from pathlib import Path
import json
from dataclasses import dataclass
from tqdm import tqdm
from transformers import AutoTokenizer
from outlines import models, generate

In [8]:
# utilities
def save_json(data, file_name):
    with open(file_name, 'w') as f:
        json.dump(data, f)

def load_json(file_name):
    with open(file_name, 'r') as f:
        return json.load(f)

def get_response(raw_output):
  return raw_output[0]['generated_text'][-1]['content']


In [4]:
# !!!!!!!! huggingface login required to use llama model (ADD TOKEN AFTER `--token`) !!!!!!!!
!huggingface-cli login --token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `general` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `general`


In [5]:
# settings
checkpoint = 'meta-llama/Llama-3.2-1B-Instruct'

In [6]:
# make directories for data and results
os.makedirs('ocr', exist_ok = True)
os.makedirs('info', exist_ok = True)

In [9]:
# upload JSON schema from `schema/schema.json`
files.upload()
schema = load_json('schema.json')
schema_string = json.dumps(schema, indent=2)

Saving schema.json to schema (1).json


In [10]:
# Upload all JSON files from `ocr_results/`
os.chdir('ocr')
ocr_paths = files.upload()
ocr_extractions = [{'key': Path(el).stem,
                    'ocr': load_json(el)} for el in ocr_paths.keys()]
os.chdir('..')

Saving 81882169_2170.json to 81882169_2170.json
Saving 81841302.json to 81841302.json
Saving 80909413.json to 80909413.json


In [11]:
# Template for information extraction
@dataclass
class Template:
  key: str
  ocr_extraction: str

  @classmethod
  def from_dict(cls, dict_):
    return cls(dict_['key'], dict_['ocr'])

  def ocr_to_string(self):
    return '\n'.join(self.ocr_extraction)

  def prompt(self):
    string = self.ocr_to_string()

    system = f'''The following is a document containing one or more emails.

Your task is to read the following emails and extract a JSON object that will capture the common structure of emails.

There may be multiple emails in the document—extract them all if so.

It is of utmost importance that you extract EXACTLY according to the JSON schema.

Now extract the JSON object.
'''

    user = string

    messages = [
      {"role": "sytem", "content": system},
      {"role": "user", "content": user},
    ]

    return messages

  def hf_to_outlines(self, messages):
    return tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = True)

  def generate(self, generator):
    messages = self.prompt()
    outlines_input = self.hf_to_outlines(messages)
    return generator(outlines_input)

In [12]:
# instantiate models
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = models.transformers(checkpoint)
generator = generate.json(model, schema_string)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [13]:
# generate records and save
for el in tqdm(ocr_extractions):
  template = Template.from_dict(el)
  info = template.generate(generator)
  save_json(info, f'info/{el["key"]}.json')

100%|██████████| 3/3 [03:25<00:00, 68.34s/it]
